In [173]:
import numpy as np

In [174]:
cars = ['1', '3', '4', '6', '7', '12', '14', '15', '17', '18', 
        '19', '20', '21', '22', '23', '24', '25', '26', '27', 
        '28', '29', '59', '60', '64', '66', '88', '98']
detectors = ['numenta', 'random', 'bayes', 'expose']
thresholds = ['0.36', '0.997', '0.39', '0.992']

In [239]:
t_i = 0
t_car = cars[0]
t_tol = 10000

In [240]:
t_labels = np.loadtxt('labels/indy2018-' + t_car + '-labels.txt')

In [241]:
t_events = np.array(list(set(t_labels.flatten())))
t_events.shape[0]

31

In [242]:
t_result = np.loadtxt('results/' + detectors[t_i] + \
                      '/anomaly_' + thresholds[t_i] + \
                      '_indy2018-' + t_car + '-vspeed.csv')

In [243]:
t_mask_n = t_result[:, 2] == 1
t_anomal = t_result[t_mask_n]
t_speeds = t_anomal[:, 0]

In [244]:
t_mask_1 = t_speeds >= t_labels[0, 0]
t_mask_2 = t_speeds <= t_labels[-1, 1]
t_mask_a = t_mask_1 & t_mask_2

In [245]:
t_middle = t_speeds[t_mask_a]
t_middle.shape[0]

167

In [246]:
t_matchs = np.sum(abs(t_middle[None, :] - \
                      t_events[:, None]) < t_tol, axis=1)
# t_falpos = np.sum(t_matchs)
t_matchs[t_matchs >= 1] = 1
t_matchs

array([1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 1])

In [247]:
t_match2 = np.sum(abs(t_middle[None, :] - \
                      t_events[:, None]) < t_tol, axis=0)
t_match2

array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 1, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [248]:
np.sum(t_match2)

104

#### combine

In [260]:
t_tol_1 = 1000
t_tol_2 = 6000

print('car, {:7s}, {:7s}, {:7s}, {:7s}'.format(\
                    detectors[0], detectors[1], detectors[2], detectors[3]))
for t_car in cars:
    t_labels = np.loadtxt('labels/indy2018-' + t_car + '-labels.txt')
    t_events = np.array(list(set(t_labels.flatten())))

    accurs = []
    falpos = []
    for t_i in range(len(detectors)):
        t_result = np.loadtxt('results/' + detectors[t_i] + \
                              '/anomaly_' + thresholds[t_i] + \
                              '_indy2018-' + t_car + '-vspeed.csv')
        t_mask_n = t_result[:, 2] == 1
        t_anomal = t_result[t_mask_n]
        t_speeds = t_anomal[:, 0]
        
        t_mask_1 = t_speeds >= t_labels[0, 0] - t_tol_1
        t_mask_2 = t_speeds <= t_labels[-1, 1] + t_tol_1
        t_mask_a = t_mask_1 & t_mask_2
        t_middle = t_speeds[t_mask_a]
        
        t_matchs = np.sum(abs(t_middle[None, :] - \
                              t_events[:, None]) < t_tol_1, axis=1)
        t_matchs[t_matchs >= 1] = 1
        t_accura = np.sum(t_matchs) / t_matchs.shape[0] * 100
        accurs.append(t_accura)

        t_match2 = np.sum(np.abs(t_middle[None, :] - \
                              t_events[:, None]) < t_tol_2, axis=0)
        t_match2[t_match2 >= 1] = 1
        t_falpos = np.sum(t_match2)
        t_fp_rat = (t_middle.shape[0] - t_falpos) / t_middle.shape[0] * 100
        falpos.append(t_fp_rat)
    
    # print('{:3s}, {:6.2f}%, {:6.2f}%, {:6.2f}%, {:6.2f}%'.format(\
    #             t_car, accurs[0], accurs[1], accurs[2], accurs[3]))
    print('{:3s}, {:6.2f}%, {:6.2f}%, {:6.2f}%, {:6.2f}%'.format(\
                t_car, falpos[0], falpos[1], falpos[2], falpos[3]))
    

car, numenta, random , bayes  , expose 
1  ,  65.87%,  99.41%,  77.78%,  60.11%
3  ,  79.17%,  99.18%,  83.85%,  69.52%
4  ,  76.43%,  97.73%,  89.58%,  64.54%
6  ,  86.10%,  97.27%,  86.30%,  73.68%
7  ,  73.98%,  97.49%,  76.10%,  77.92%
12 ,  61.59%,  98.48%,  81.14%,  81.12%
14 ,  74.49%,  97.21%,  87.38%,  85.80%
15 ,  84.71%,  98.67%,  85.50%,  94.96%
17 ,  76.86%,  96.46%,  84.50%,  77.04%
18 ,  77.24%,  97.56%,  77.40%,  59.41%
19 ,  68.07%,  95.96%,  79.18%,  63.78%
20 ,  68.75%,  98.56%,  76.14%,  54.55%
21 ,  72.56%,  97.38%,  76.35%,  62.42%
22 ,  70.91%,  98.58%,  83.08%,  71.24%
23 ,  70.95%,  95.85%,  79.46%,  67.37%
24 ,  76.26%,  98.29%,  83.71%,  73.77%
25 ,  49.24%,  99.44%,  83.13%,  74.07%
26 ,  77.05%,  98.45%,  78.89%,  83.74%
27 ,  88.50%,  95.12%,  85.51%,  75.91%
28 ,  87.12%,  98.31%,  87.22%,  80.00%
29 ,  88.50%,  96.72%,  88.42%,  78.68%
59 ,  78.85%,  99.00%,  85.86%,  84.08%
60 ,  83.82%,  95.61%,  86.23%,  79.27%
64 ,  80.25%,  98.64%,  85.95%,  79.33%
